Still TODO:

* Hash Out Training/Validation/Testing Accuracy Metrics Better
  * Top N Accuracy on Dataset
  * DNA Comparison


Need to finish the code first:

* Finish the paper
* Finish the presentation 

In [1]:
## Directory Packges
import os
from glob import glob
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## Data Packages ##
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
from tqdm import tqdm

## Tensorflow ##
import tensorflow as tf
from tensorflow import keras

Mounted at /content/drive


In [3]:
## Pathings ##
PATH = "drive/My Drive/dog-breed-identification/"
train_path = os.path.join(PATH, "train/*")
test_path = os.path.join(PATH, "test/*")
labels_path = os.path.join(PATH, "labels.csv")
models_path = PATH + 'models/'

## Image Parameters ##
size = 224
num_classes = 120
batch = 36

In [4]:
## Load Data Functions##

def get_labels(ids, labels_df, breed2id):
  
  labels = []
  for image_id in ids:
      image_id = image_id.split("/")[-1].split(".")[0]
      breed_name = list(labels_df[labels_df.id == image_id]["breed"])[0]
      breed_idx = breed2id[breed_name]
      labels.append(breed_idx)

  return labels

def read_image(path, size):

    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (size, size))
    image = image / 255.0
    image = image.astype(np.float32)

    return image

def parse_data(x, y):

    x = x.decode()

    image = read_image(x, size)
    label = [0] * num_classes
    label[y] = 1
    label = np.array(label)
    label = label.astype(np.int32)

    return image, label

def tf_parse(x, y):

    x, y = tf.numpy_function(parse_data, [x, y], [tf.float32, tf.int32])
    x.set_shape((224, 224, 3))
    y.set_shape((120))

    return x, y

def tf_dataset(x, y, batch=8):
  
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()

    return dataset

def build_model(size, num_classes):

    # Define Input Images
    inputs = keras.layers.Input((size, size, 3))

    # Using MobileNetV2 as Backbone of The Model
    backbone = keras.applications.MobileNetV2(
        input_tensor=inputs, include_top=False, weights="imagenet"
    )
    backbone.trainable = True
    
    # Adding Some Addition Layers
    x = backbone.output
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(1024, activation="relu")(x)
    x = keras.layers.Dense(num_classes, activation="softmax")(x)

    # Combine for Final Model
    model = keras.Model(inputs, x)

    return model

In [5]:
# Load Lables
train_labels_df = pd.read_csv(labels_path)

# Create Lookup for Breed Name + ID
breed = train_labels_df["breed"].unique()
breed2id = {name: i for i, name in enumerate(breed)}
id2breed = {i: name for i, name in enumerate(breed)}

# Load Image IDs
train_ids = glob(train_path)

# Get Lables
train_labels = get_labels(train_ids, train_labels_df, breed2id)

# Split the Data into Training + Validtion
train_x, valid_x = train_test_split(train_ids, test_size=0.2, random_state=42)
train_y, valid_y = train_test_split(train_labels, test_size=0.2, random_state=42)

# Fully Load Data in TF Dataset
train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

In [6]:
# Load Trained Model - Normal
model = build_model(size, num_classes)
model.load_weights(models_path + "model_normal.h5")

# # Load Trained Model - CutMix
# model = build_model(size, num_classes)
# model.load_weights(models_path + "model_cutmix.h5")

9406464/9406464 [==============================] - 0s 0us/step


In [7]:
# Look at First 10 in Validation Data
acc_1 = 0 
acc_3 = 0
acc_5 = 0

for i, path in tqdm(enumerate(valid_x)):

    # Load Image
    image = read_image(path, 224)
    image = np.expand_dims(image, axis=0)

    # Predict Top 5 Breeds by Probability
    pred = model.predict(image)[0]
    label_idx = list(np.argsort(pred)[-1:-6:-1])

    # Store Results
    results = [
        {'breed': id2breed[idx], 'probability': pred[idx]}
        for idx in label_idx
    ]
    print(results)

    # Get True Label
    ori_breed = id2breed[valid_y[i]]

    #add to accuracy if true label is in top 5 breeds
    if any(key['breed'] == ori_breed for key in results[:1]):
        acc_1 += 1
        acc_3 += 1
        acc_5 += 1
    elif any(key['breed'] == ori_breed for key in results[:3]):
        acc_3 += 1
        acc_5 += 1
    elif any(key['breed'] == ori_breed for key in results[:5]):
        acc_5 += 1


    print(ori_breed)


0it [00:00, ?it/s]

1/1 [==============================] - 2s 2s/step


1it [00:02,  2.10s/it]

[{'breed': 'vizsla', 'probability': 0.9947015}, {'breed': 'redbone', 'probability': 0.0025420352}, {'breed': 'bloodhound', 'probability': 0.0023898473}, {'breed': 'rhodesian_ridgeback', 'probability': 0.00014154961}, {'breed': 'basset', 'probability': 0.000118788535}]
vizsla
1/1 [==============================] - 0s 55ms/step


2it [00:02,  1.17s/it]

...

[{'breed': 'flat-coated_retriever', 'probability': 0.5609344}, {'breed': 'labrador_retriever', 'probability': 0.19909956}, {'breed': 'kelpie', 'probability': 0.08206229}, {'breed': 'newfoundland', 'probability': 0.06765834}, {'breed': 'chesapeake_bay_retriever', 'probability': 0.031129569}]
flat-coated_retriever
1/1 [==============================] - 0s 89ms/step


307it [04:47,  7.29it/s]

[{'breed': 'bedlington_terrier', 'probability': 0.9997012}, {'breed': 'standard_poodle', 'probability': 0.0001873932}, {'breed': 'borzoi', 'probability': 3.9581104e-05}, {'breed': 'wire-haired_fox_terrier', 'probability': 1.0919458e-05}, {'breed': 'weimaraner', 'probability': 9.761459e-06}]
bedlington_terrier


307it [04:47,  1.07it/s]


KeyboardInterrupt: ignored

In [8]:
print(f'Accuracy 1: {round(acc_1/len(valid_x), 4)}')
print(f'Accuracy 3: {round(acc_3/len(valid_x), 4)}') 
print(f'Accuracy 5: {round(acc_5/len(valid_x), 4)}') 

Accuracy 1: 0.1095
Accuracy 3: 0.1384
Accuracy 5: 0.1433


In [9]:
mixed_breeds = os.path.join(PATH, "mix-breed-pics/*")
mixed_ids = sorted(glob(mixed_breeds))

In [10]:
# Look at First 10 in Validation Data
for i, path in tqdm(enumerate(mixed_ids)):

    # Load Image
    image = read_image(path, 224)
    image = np.expand_dims(image, axis=0)

    # Predict Top 5 Breeds by Probability
    pred = model.predict(image)[0]
    print('\n\n')
    print(path)

    label_idx = list(np.argsort(pred)[-1:-6:-1])
  
    # Store Results
    results = [
        {'breed': id2breed[idx], 'probability': pred[idx]}
        for idx in label_idx
    ]

    print(results)

    print('\n')

0it [00:00, ?it/s]

1/1 [==============================] - 0s 63ms/step


1it [00:00,  3.25it/s]




drive/My Drive/dog-breed-identification/mix-breed-pics/IMG_2094.jpg
[{'breed': 'chihuahua', 'probability': 0.7569966}, {'breed': 'pembroke', 'probability': 0.06659704}, {'breed': 'mexican_hairless', 'probability': 0.055058975}, {'breed': 'schipperke', 'probability': 0.0419409}, {'breed': 'dhole', 'probability': 0.015065323}]


1/1 [==============================] - 0s 56ms/step


2it [00:00,  3.00it/s]

...




drive/My Drive/dog-breed-identification/mix-breed-pics/val_39.jpg
[{'breed': 'pug', 'probability': 0.9687396}, {'breed': 'brabancon_griffon', 'probability': 0.016008338}, {'breed': 'french_bulldog', 'probability': 0.002212945}, {'breed': 'boxer', 'probability': 0.0021174538}, {'breed': 'bull_mastiff', 'probability': 0.0021114687}]


1/1 [==============================] - 0s 67ms/step


42it [00:20,  2.82it/s]




drive/My Drive/dog-breed-identification/mix-breed-pics/val_40.jpg
[{'breed': 'american_staffordshire_terrier', 'probability': 0.3299659}, {'breed': 'great_dane', 'probability': 0.20135488}, {'breed': 'weimaraner', 'probability': 0.18954872}, {'breed': 'italian_greyhound', 'probability': 0.10370922}, {'breed': 'doberman', 'probability': 0.044472653}]


1/1 [==============================] - 0s 60ms/step


43it [00:20,  2.08it/s]




drive/My Drive/dog-breed-identification/mix-breed-pics/val_41.jpg
[{'breed': 'labrador_retriever', 'probability': 0.4708582}, {'breed': 'miniature_pinscher', 'probability': 0.23087879}, {'breed': 'black-and-tan_coonhound', 'probability': 0.095201135}, {'breed': 'rottweiler', 'probability': 0.09507035}, {'breed': 'appenzeller', 'probability': 0.04058035}]


